In [54]:
from scipy.signal import hilbert
from scipy.signal import welch
from scipy.signal import butter, lfilter,firls
from scipy import signal
import sys
import os
# setting path
#sys.path.append('../')
#from utils import METHODS_EOS
import pandas as pd
from mne.time_frequency import psd_array_multitaper, psd_array_welch
from fooof import FOOOF
import numpy as np
import argparse
import mne.io
import mne
import multiprocessing as mp
from scipy.io import loadmat
from scipy.io import savemat
import glob
from joblib import Parallel, delayed
import multiprocessing as mp  # You can keep this if needed elsewhere

import pandas as pd
import numpy as np
import scipy
import neurokit2 as nk
from numpy import (absolute, angle, arctan2,
                   diff, exp, imag, mean, real, square,
                   var)
from scipy.signal import hilbert
from scipy.io import loadmat

def pcf(data):
    """Estimate the pair correlation function (PCF) in a network of
    oscillators, equivalent to the susceptibility in statistical physics.
    The PCF shows a sharp peak at a critical value of the coupling between
    oscillators, signaling the emergence of long-range correlations between
    oscillators.

    Parameters
    ----------
    data : 2d array
        The filtered input data, where the first dimension is the different
        oscillators, and the second dimension is time.

    Returns
    -------
    pcf : float
        The pair correlation function, a scalar value >= 0.
    orpa: float
        Absolute value of the order parameter (degree of synchronicity)
        averaged over time, being equal to 0 when the oscillators’ phases are
        uniformly distributed in [0,2π ) and 1 when they all have the same
        phase.
    orph_vector: 1d array (length = N_timepoints)
        Order parameter phase for every moment in time.
    orpa_vector: 1d array (length = N_timepoints)
        Absolute value of the order parameter (degree of synchronicity) for
        every moment in time.

    References
    ----------
    Yoon et al. (2015) Phys Rev E 91(3), 032814.
    """

    N_ch = min(data.shape)  # Nr of channels
    N_time = max(data.shape)  # Nr of channels

    # inifialize empty array
    inst_phase = np.zeros(data.shape)
    z_vector = []

    # calculate Phase of
    for i in range(N_ch):
        inst_phase[i,:] = np.angle(scipy.signal.hilbert(data[i,:]))

    for t in range(N_time):
        # get global synchronization order parameter z over time
        z_vector.append(np.mean(exp(1j * inst_phase[:,t])))

    z_vector = np.array(z_vector)

    #  r =|z| degree of synchronicity
    orpa_vector = abs(z_vector)
    # get order phases
    orph_vector = arctan2(imag(z_vector), real(z_vector))

    # get PCF = variance of real part of order parameter
    # var(real(x)) == (mean(square(real(x))) - square(mean(real(x))))
    pcf = N_ch * var(real(z_vector))
    # time-averaged Order Parameter
    orpa = mean(orpa_vector);

    return pcf, orpa, orph_vector, orpa_vector



def calculate_values(trial, epochs):
    if len(epochs)==1:
        data_tr = epochs.get_data()[0]
    else:
        data_tr = epochs[trial].get_data()[0]

    #PLE = Methods_EOS.ple(data_tr, m = 5, tau = 2)
    #PLI = Methods_EOS.pli(data_tr)
    PCF, OR_mean, orph_vector_tr, orpa_vector_tr = pcf(data_tr)
    #print(f'done Trial {str(trial)}')

    #return PLI, PLE, PCF, OR_mean, OR_var
    return PCF, OR_mean


In [ ]:
out_dir = 'data/output/EOS/'
in_dir = 'data/input/10s/'

fbands = [[1,45], [1, 4], [4, 8], [8, 13], [13, 30], [30, 45]]

for fband in fbands: 
    minfreq = fband[0]
    maxfreq = fband[1]

    # output
    PCF_mean = []
    OR_mean = []
    sub_all = []
    day_all = []   
    condition_all = []

    # make output directory
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # load patient info and conditions
    paths = glob.glob(f'{in_dir}*.fif')    
    paths.sort()


    #loop over all conditions and particiants
    for i, path in enumerate(paths):
        basename = os.path.basename(path)
        sub = basename[:4]
        day = basename[5:9]
        condition = basename[10:-8]
        if len(basename)==36:
            condition = basename[10:15]+basename[-5:-4]


        print(f"Analyzing Synchrony of {path}");

        #################################
        #          LOAD  DATA          #
        #################################

        epochs_mne = mne.read_epochs(path, preload=True)
        epochs_mne.interpolate_bads(reset_bads=True)
        epochs_mne.pick_types(eeg=True, meg=False, stim=False, eog=False, ecg=False, emg=False, misc=False, exclude='bads')

        epochs = epochs_mne.get_data()[:,:32,:]
        fs = 256 #check this


        # prepare data
        #epochs_res = epochs_mne.resample(250)
        epochs_filt = epochs_mne.filter(minfreq, maxfreq, verbose = False)

        # if data is too long only use the first 3 min of data
        nr_trials = min([len(epochs_filt),80]);  #ADJUST EPOCHS!!!! check this
        nr_channels =  epochs_filt.info['nchan']

        ###############################################
        #    Calculate Pair Correlation Function     #
        ###############################################

        pool = mp.Pool(mp.cpu_count())
        # loop over every time segment

        # prepare input for parallel function
        input = []
        
        ## two alternative input generators
        for trial in range(nr_trials):
            input.append((trial,epochs_filt))

        # for trial in range(nr_trials):
        #     epoch_selected = epochs_filt[trial]
        #     input.append((trial,epoch_selected))

        num_cores = mp.cpu_count()
        results = Parallel(n_jobs=num_cores)(
            delayed(calculate_values)(*args) for args in input
        )

        '''
        pool = mp.Pool(mp.cpu_count())
        results = pool.starmap(calculate_values,input)
        pool.close()
        '''
        
        PCF_mean.append(np.median(pd.DataFrame(results)[0]))
        OR_mean.append(np.median(pd.DataFrame(results)[1]))
        sub_all.append(sub)
        day_all.append(day)
        condition_all.append(condition)

        # save dataframe
        output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all, 'PCF_mean':PCF_mean,'OR_mean':OR_mean}
        output_df = pd.DataFrame(output_df)
        output_df.to_csv(f'{out_dir}/EOS_{minfreq}_{maxfreq}.csv', index=False, sep=',')


    # save dataframe
    output_df = {'sub':sub_all, 'day': day_all,'condition':condition_all, 'PCF_mean':PCF_mean,'OR_mean':OR_mean}
    output_df = pd.DataFrame(output_df)
    output_df.to_csv(f'{out_dir}/EOS_{minfreq}_{maxfreq}.csv', index=False, sep=',')



Analyzing Synchrony of data/input/10s/sub0-day1-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/1927387291.py:44: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:

Analyzing Synchrony of data/input/10s/sub0-day1-mindfulness-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub0-day1-mindfulness-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of

Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

Analyzing Synchrony of data/input/10s/sub5-day4-jhana-epo.fif
Reading /Users/jonasmago/PhD_code_data/github/Criticality_PCI_Anesthesia/data/input/10s/sub5-day4-jhana-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    9996.09 ms
        0 CTF compensation matrices available


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/161624069.py:101: FutureWarning: The current default of

Not setting metadata
140 matching events found
No baseline correction applied
0 projection items activated
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.7 mm
Computing interpolation matrix from 29 sensor positions
Interpolating 3 sensors
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/1927387291.py:44: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs = epochs_mne.get_data()[:,:32,:]


KeyboardInterrupt: 

In [ ]:
        num_cores = mp.cpu_count()
        results = Parallel(n_jobs=num_cores)(
            delayed(calculate_values)(*args) for args in input
        )


In [42]:
input[2][1].get_data().shape

/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_81309/3406732621.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  input[2][1].get_data().shape


(92, 32, 2560)

In [ ]:
def calculate_values(trial, epochs):

    data_tr = epochs[trial].get_data()[0]

    #PLE = Methods_EOS.ple(data_tr, m = 5, tau = 2)
    #PLI = Methods_EOS.pli(data_tr)
    PCF, OR_mean, orph_vector_tr, orpa_vector_tr = pcf(data_tr)
    #print(f'done Trial {str(trial)}')

    #return PLI, PLE, PCF, OR_mean, OR_var
    return PCF, OR_mean


In [23]:
input

[(0,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (1,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (2,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (3,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (4,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (5,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (6,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (7,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (8,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (9,
  <EpochsFIF |

In [ ]:
   for trial in range(nr_trials):
            input.append((trial,epochs_filt))

        # for trial in range(nr_trials):
        #     input.append((trial, epochs_filt[trial:trial+1]))  # Select only one epoch

        num_cores = mp.cpu_count()
        results = Parallel(n_jobs=num_cores)(
            delayed(calculate_values)(*args) for args in input
        )


In [ ]:
     nr_trials = min([len(epochs_filt),80]);  #ADJUST EPOCHS!!!! check this
        nr_channels =  epochs_filt.info['nchan']

        ###############################################
        #    Calculate Pair Correlation Function     #
        ###############################################

        pool = mp.Pool(mp.cpu_count())
        # loop over every time segment

        # prepare input for parallel function
        input = []
        for trial in range(nr_trials):
            input.append((trial,epochs_filt))


In [30]:
input

[(0,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (1,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (2,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (3,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (4,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (5,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (6,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (7,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (8,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (9,
  <EpochsFIF |

In [33]:
sys.getsizeof(input[0][1])

48

In [18]:
for trial in range(nr_trials):
    input.append((trial,epochs_filt[trial]))

In [15]:
 range(nr_trials)

range(0, 80)

In [11]:
input[1]

(1,
 <EpochsFIF |  100 events (all good), 0 – 9.99609 s, baseline off, ~62.6 MB, data loaded,
  '1': 100>)

In [45]:
epochs_filt[1]

Number of events,1
Events,1: 1
Time range,0.000 – 9.996 s
Baseline,off


In [49]:
for trial in range(nr_trials):
    epoch_selected = epochs_filt[trial]
    input.append((trial,epoch_selected))

In [53]:
len(input[0][1])

1

In [13]:
input[4]

(4,
 <EpochsFIF |  100 events (all good), 0 – 9.99609 s, baseline off, ~62.6 MB, data loaded,
  '1': 100>)

In [43]:
input

[(0,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (1,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (2,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (3,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (4,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (5,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (6,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (7,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (8,
  <EpochsFIF |  92 events (all good), 0 – 9.99609 s, baseline off, ~57.6 MB, data loaded,
   '1': 92>),
 (9,
  <EpochsFIF |

In [ ]:
num_cores = mp.cpu_count()
results = Parallel(n_jobs=num_cores)(
    delayed(calculate_values)(*args) for args in input
)